In [39]:
import cv2
import tkinter as tk
from tkinter import filedialog
from tkinter.filedialog import askopenfile
from PIL import Image, ImageTk
import skimage
import numpy as np

window = tk.Tk()
window.geometry("600x350")
window.columnconfigure(0, weight=4) 
window.columnconfigure(1, weight=4)
window.title('Image Cropping')

label_title = tk.Label(window,text='              Image',font=('times', 16))
label_title.grid(row=0, column=0, pady=(20, 10), padx=(20, 0), sticky="w")

button_upload = tk.Button(window, text='Choose File', width=27, command=lambda:upload_file()) 
button_upload.grid(row=1, column=0, pady=(0,20), padx=(20, 0), sticky="w")

label_cropping = tk.Label(window,text='           Cropping',font=('times', 16))
label_cropping.grid(row=0, column=1, rowspan=1, pady=(20, 10), padx=(20, 0), sticky="w")

def upload_file():
    global label_image_upload
    global label_image_cropping

    try:
        label_image_upload.grid_remove()
        label_image_cropping.grid_remove()
    except:
        print("")
    finally:  
        #Image Upload
        image_type = [('Image File', ['*.jpg', '*.jpeg', '*.png'])]
        image_name = filedialog.askopenfilename(filetypes=image_type)
        image_upload_open = Image.open(image_name)
        image_upload_resize = image_upload_open.resize((197, 197), Image.Resampling.LANCZOS)
        image_upload = ImageTk.PhotoImage(image_upload_resize)
        label_image_upload = tk.Label(window, image=image_upload)
        label_image_upload.image = image_upload
        label_image_upload.grid(row=2, column=0, pady=(0,20), padx=(20, 0), sticky="w")

        #Image Property
        image = cv2.imread(image_name, cv2.IMREAD_UNCHANGED)
        w = image.shape[1]
        h = image.shape[0]
        mode = {'1':1, 'L':8, 'P':8, 'RGB':24, 'RGBA':32, 'CMYK':32, 'YCbCr':24, 'I':32, 'F':32}
        image_depth = mode[image_upload_open.mode]

        #Image Cropping
        xR = 100 #x Right
        xL = 20 #x Left
        yT = 50 #y Top
        yB = 180 #y Bottom
        image_cropping = skimage.io.imread(fname=image_name)
        image_cropping_temp = np.array(image_cropping, copy=True)
        if image_depth == 8 or image_depth == 24:
            x = 0
            y = 0
            x_count = 0
            y_count = 0
            #Cropping y / Row
            while True:
                if y_count == h-1:
                    break
                if not (y >= yT and y <= yB):
                    image_cropping = np.delete(image_cropping, y, axis=0)
                    y_count+=1
                    yT-=1
                    yB-=1
                else:
                    y+=1
                    y_count+=1
            #Cropping x / Column
            while True:
                if x_count == w-1:
                    break
                if not (x >= xL and x <= xR):
                    image_cropping = np.delete(image_cropping, x, axis=1)
                    x_count+=1
                    xR-=1
                    xL-=1
                else:
                    x+=1
                    x_count+=1
        image_cropping = Image.fromarray(image_cropping)
        image_cropping_resize = image_cropping.resize((197, 197), Image.Resampling.LANCZOS)
        image_cropping = ImageTk.PhotoImage(image_cropping_resize)
        label_image_cropping = tk.Label(window, image=image_cropping)
        label_image_cropping.image = image_cropping
        label_image_cropping.grid(row=2, column=1, pady=(0,20), padx=(20, 0), sticky="w")

window.mainloop()